In [2]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 460.5 kB/s eta 0:00:001m681.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:003.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:004.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 937.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 5.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.9 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 29.2 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 19.9 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
model_name = 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
query = 'I just discovered the course. Can I still join it?'

In [7]:
vector_embedding = model.encode(query)

In [8]:
first_value = vector_embedding[0]

print("First value of the vector embedding:", first_value)

First value of the vector embedding: 0.078222655


In [9]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'

docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for doc in documents_raw:
    if doc.get('course') == 'machine-learning-zoomcamp':
        documents.append(doc)

In [10]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [11]:
len(documents)

375

In [13]:
import numpy as np
from tqdm.auto import tqdm

In [14]:
embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(model.encode(qa_text))
    
X = np.array(embeddings)

print(X.shape)

  0%|          | 0/375 [00:00<?, ?it/s]

(375, 768)


In [16]:
scores = X.dot(vector_embedding)
highest_score = np.max(scores)

print("Highest Cosine Similarity Score:", highest_score)

Highest Cosine Similarity Score: 0.6506573


In [18]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
results = search_engine.search(vector_embedding, num_results=5)
print("Search Results:", results)

Search Results: [{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp', 'id': 'ee58a693'}, {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nO

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [20]:
def calculate_hitrate(ground_truth, search_engine, num_results):
    cnt = 0
    total = len(ground_truth)

    for record in tqdm(ground_truth, desc="Calculating hitrate"):
        query_vector = model.encode(record['question'])
        results = search_engine.search(query_vector, num_results)
        if record['document'] in [result['id'] for result in results]:
            cnt += 1
    
    hitrate = cnt / total
    return hitrate

hitrate = calculate_hitrate(ground_truth, search_engine, num_results=5)
print(f"Hit-rate: {hitrate}")

Calculating hitrate:   0%|          | 0/1830 [00:00<?, ?it/s]

Hit-rate: 0.9398907103825137


In [21]:
!docker run \
    --rm \
    --name elasticsearch \
    -d \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Unable to find image 'docker.elastic.co/elasticsearch/elasticsearch:8.4.3' locally
8.4.3: Pulling from elasticsearch/elasticsearch

bc8cf243: Pulling fs layer 
2014c641: Pulling fs layer 
b900b382: Pulling fs layer 
ca3684dd: Pulling fs layer 
39673d83: Pulling fs layer 
8e9af4ef: Pulling fs layer 
ca40e65f: Pulling fs layer 
cbaa0d4a: Pulling fs layer 
dbaccd81: Pull complete 8.1kB/108.1kBBDigest: sha256:739ec9d428869f16e9e02247d5082849ebb4302c87e0abf9f70971cbb40c3bab
Status: Downloaded newer image for docker.elastic.co/elasticsearch/elasticsearch:8.4.3
31982a1291eedbf1b4be554213a4891c12b72493d94340993e83974e36e07282


In [23]:
pip install elasticsearch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 2.4 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '31982a1291ee', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'yY5ot4ByR1uNo1y-o_2rSw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [26]:
!curl -X GET "localhost:9200/course-questions/_mapping"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"course-questions":{"mappings":{"properties":{"course":{"type":"keyword"},"id":{"type":"keyword"},"question":{"type":"text"},"question_text_vector":{"type":"dense_vector","dims":768,"index":true,"similarity":"cosine"},"question_vector":{"type":"dense_vector","dims":768,"index":true,"similarity":"cosine"},"section":{"type":"text"},"text":{"type":"text"},"text_vector":{"type":"dense_vector","dims":768,"index":true,"similarity":"cosine"}}}}}

In [27]:
for doc, embedding in zip(tqdm(documents, desc="Adding embeddings and indexing"), embeddings):
    try:
        doc["question_vector"] = embedding.tolist()  
        doc["text_vector"] = embedding.tolist()
        doc["question_text_vector"] = embedding.tolist()
        
        # print("Indexing document:", doc)

        es_client.index(index="course-questions", document=doc)
    except Exception as e:
        print(f"Error processing document: {e}")

Adding embeddings and indexing:   0%|          | 0/375 [00:00<?, ?it/s]

In [28]:
index_mapping = es_client.indices.get_mapping(index=index_name)
print(index_mapping[index_name]['mappings'])

{'properties': {'course': {'type': 'keyword'}, 'id': {'type': 'keyword'}, 'question': {'type': 'text'}, 'question_text_vector': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'cosine'}, 'question_vector': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'cosine'}, 'section': {'type': 'text'}, 'text': {'type': 'text'}, 'text_vector': {'type': 'dense_vector', 'dims': 768, 'index': True, 'similarity': 'cosine'}}}


In [29]:
def elastic_search(field, vector, num_results):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": num_results,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["id", "text", "section", "question", "course"]  
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append({
            "text": hit['_source']['text'],
            "section": hit['_source'].get('section', 'Unknown'),  
            "question": hit['_source'].get('question', 'Unknown'),
            "course": hit['_source'].get('course', 'Unknown'),
            "id": hit['_source'].get('id') 
        })
    
    return result_docs

In [30]:
elastic_search("question_vector", model.encode(query), num_results=1)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'}]

In [31]:
[doc for doc in documents if doc["id"] == "ee58a693"]

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_vector': [0.0806286409497261,
   -0.06663887947797775,
   0.02527306228876114,
   -0.013004408217966557,
   0.07587282359600067,
   -0.05946270748972893,
   -0.02188383787870407,
   0.0029000400099903345,
   0.0007928351988084614,
   -0.0052224150858819485,
   -0.03365179896354675,
   -0.027913514524698257,
   0.05811655521392822,
   0.039748333394527435,
   0.05441855639219284,
   -0.038251712918281555,
   0.06305737

In [32]:
def calculate_hitrate(ground_truth, search_engine, num_results):
    cnt = 0
    total = len(ground_truth)

    for record in tqdm(ground_truth, desc="Calculating hitrate"):
        query_vector = model.encode(record['question'])
        results = search_engine("question_vector", query_vector, num_results)
        if record['document'] in [result['id'] for result in results]:
            cnt += 1
    
    hitrate = cnt / total
    return hitrate

hitrate = calculate_hitrate(ground_truth, search_engine=elastic_search, num_results=5)
print(f"Hit-rate: {hitrate}")

Calculating hitrate:   0%|          | 0/1830 [00:00<?, ?it/s]

Hit-rate: 0.9398907103825137
